In [ ]:
import copy
import time
import sys
import tempfile
import os
from pathlib import Path
import logging
import datetime
import threading
import queue
import cProfile
import functools

#import dask
#from dask import array
import numpy
#from dask.distributed import Client, performance_report
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
#import ml_collections
import parmed
import warnings
from plip.structure.preparation import PDBComplex

import fegrow

#import al_for_fep.models.sklearn_gaussian_process_model

# log = logging.getLogger(__name__)
# logging.basicConfig(level=logging.INFO)

# try:
#     from mycluster import create_cluster
# except ImportError:
#     def create_cluster():
#         from dask.distributed import LocalCluster
#         return LocalCluster()


# xstal interactions from 24 mpro fragments
mpro_crystal_structures_interactions = {'hacceptor_THR_25_8',
                                         'hdonor_THR_25_8',
                                         'hydrophobic_THR_25',
                                         'hacceptor_HIS_41_8',
                                         'hydrophobic_HIS_41',
                                         'pistacking_HIS_41',
                                         'hacceptor_CYS_44_8',
                                         'hydrophobic_PHE_140',
                                         'hacceptor_ASN_142_8',
                                         'hdonor_ASN_142_7',
                                         'hdonor_GLY_143_7',
                                        'hacceptor_SER_144_8',
                                        'hdonor_SER_144_7',
                                        'hdonor_SER_144_8',
                                        'hdonor_CYS_145_7',
                                        'hacceptor_HIS_163_7',
                                        'hydrophobic_MET_165',
                                        'hacceptor_GLU_166_8',
                                        'hdonor_GLU_166_7',
                                        'hdonor_GLU_166_8',
                                        'hydrophobic_GLU_166',
                                        'saltbridge_GLU_166',
                                        'hydrophobic_PRO_168',
                                        'hydrophobic_ASP_187',
                                        'hacceptor_GLN_189_8',
                                        'hdonor_GLN_189_7',
                                        'hydrophobic_GLN_189'}


def plip_mpro_merge_score(protein, ligand):
    with tempfile.TemporaryDirectory() as TD:

        # turn the rdkit.Mol into an SDF file
        if isinstance(ligand, Chem.Mol):
            ligand_path = os.path.join(TD, "ligand.sdf")
            with Chem.SDWriter(ligand_path) as SD:
                SD.write(ligand)
                ligand = ligand_path

        lig = parmed.load_file(ligand)
        if isinstance(lig, list):
            warnings.warn("The ligand was an array (SDF?). Using the first frame. ")
            lig = lig[0]
        protein = parmed.load_file(protein)
        system = protein + lig
        complex_path = os.path.join(TD, "complex.pdb")
        system.save(complex_path, renumber=False)
        return plip_mpro_score(str(complex_path))

def plip_mpro_score(complex_path):
    """
    Get all the protein interactions from the interactions between the protein and the ligand

    Args:
        complex_path:

    Returns: A list of strings
    """
    complex = PDBComplex()
    complex.load_pdb(complex_path) # Load the PDB file into PLIP class
    complex.analyze()

    # assume there is only one ligand for now
    if len(complex.interaction_sets) != 1:
        raise ValueError("PLIP detected more (or less) than one ligand?!")

    # pair key and values
    interactions = list(complex.interaction_sets.values())[0]

    # take all the interactions
    hydrophobic_contacts = ["hydrophobic_" + c.restype + "_" + str(c.resnr) for c in interactions.hydrophobic_contacts]
    # extract protein donors
    hdonors = ["hdonor_" + d.restype + "_" + str(d.resnr) + "_" + str(d.d.atomicnum) for d in interactions.hbonds_pdon]
    # extract protein acceptors
    hacceptors = ["hacceptor_" + a.restype + "_" + str(a.resnr) + "_" + str(a.a.atomicnum) for a in interactions.hbonds_ldon]
    pistacking = ["pistacking_" + r.restype + "_" + str(r.resnr) for r in interactions.pistacking]
    saltbridge = ["saltbridge_" + r.restype + "_" + str(r.resnr) for r in  interactions.saltbridge_pneg]
    waterbridge = ["waterbridge_" + r.restype + "_" + str(r.resnr) for r in interactions.water_bridges]
    pication = ["pication_" + r.restype + "_" + str(r.resnr) for r in interactions.pication_paro]
    halogen_bond = ["halogenbond_" + r.restype + "_" + str(r.resnr) for r in interactions.halogen_bonds]
    metal_complex = ["metalcomplex_" + r.restype + "_" + str(r.resnr) for r in interactions.metal_complexes]

    protein_interaction_fingerprints = (hydrophobic_contacts +
                                        hdonors +
                                        hacceptors +
                                        pistacking +
                                        saltbridge +
                                        waterbridge +
                                        pication +
                                        halogen_bond +
                                        metal_complex)

    intersection = len(mpro_crystal_structures_interactions.intersection(protein_interaction_fingerprints))
    count = len(mpro_crystal_structures_interactions) + len(protein_interaction_fingerprints)
    tanimoto_distance = intersection / (count - intersection)

    return protein_interaction_fingerprints, tanimoto_distance

mpro_crystal_structures_interactions = {'hacceptor_THR_25_8',
                                         'hdonor_THR_25_8',
                                         'hydrophobic_THR_25',
                                         'hacceptor_HIS_41_8',
                                         'hydrophobic_HIS_41',
                                         'pistacking_HIS_41',
                                         'hacceptor_CYS_44_8',
                                         'hydrophobic_PHE_140',
                                         'hacceptor_ASN_142_8',
                                         'hdonor_ASN_142_7',
                                         'hdonor_GLY_143_7',
                                        'hacceptor_SER_144_8',
                                        'hdonor_SER_144_7',
                                        'hdonor_SER_144_8',
                                        'hdonor_CYS_145_7',
                                        'hacceptor_HIS_163_7',
                                        'hydrophobic_MET_165',
                                        'hacceptor_GLU_166_8',
                                        'hdonor_GLU_166_7',
                                        'hdonor_GLU_166_8',
                                        'hydrophobic_GLU_166',
                                        'saltbridge_GLU_166',
                                        'hydrophobic_PRO_168',
                                        'hydrophobic_ASP_187',
                                        'hacceptor_GLN_189_8',
                                        'hdonor_GLN_189_7',
                                        'hydrophobic_GLN_189'}

os.chdir('/home/cree/code/gal/cs50k/enamine_study/sars-cov-2-main-protease-al-study-plip')

i_dict = {}
struc_dir = './plip_tests'
for i in [1,5,6,10]:
    i_dict[str(i)] = itrns, _ = plip_mpro_score(struc_dir+f'/{i}_complexh.pdb')

i_dict

In [8]:
import glob
os.chdir('./plip_tests/extracting_reference/')
pdbs = glob.glob('*pdb')
len(pdbs)

23

In [ ]:
x_dict = {}
os.chdir('/home/cree/code/gal/cs50k/enamine_study/sars-cov-2-main-protease-al-study-plip/plip_tests/extracting_reference/')
pdbs = glob.glob('*pdb')
os.chdir('/home/cree/code/gal/cs50k/enamine_study/sars-cov-2-main-protease-al-study-plip/plip_tests/extracting_reference')
struc_dir = './plip_tests'
for pdb in pdbs:
    x_dict[str(pdb)] = itrns, _ = plip_mpro_score(pdb)

In [ ]:
dict_values = x_dict.values()

flattened_values = [] # list to hold flattened values

# loop through each tuple in the dictionary values
for value in x_dict.values():
    # extend the list with the list part of the tuple
    flattened_values.extend(value[0])
    # append the list with the numerical part of the tuple
    flattened_values.append(value[1])

set(flattened_values)

In [47]:
ref_i = ['hacceptor_ASN_142_8',
 'hacceptor_CYS_44_8',
 'hacceptor_GLN_189_8',
 'hacceptor_GLU_166_8',
 'hacceptor_HIS_163_7',
 'hacceptor_HIS_41_8',
 'hacceptor_SER_144_8',
 'hdonor_ASN_142_7',
 'hdonor_CYS_145_7',
 'hdonor_GLN_189_7',
 'hdonor_GLU_166_7',
 'hdonor_GLU_166_8',
 'hdonor_GLY_143_7',
 'hdonor_SER_144_7',
 'hdonor_SER_144_8',
 'hdonor_THR_25_8',
 'hydrophobic_ASP_187',
 'hydrophobic_GLN_189',
 'hydrophobic_GLU_166',
 'hydrophobic_HIS_41',
 'hydrophobic_MET_165',
 'hydrophobic_PHE_140',
 'hydrophobic_PRO_168',
 'hydrophobic_THR_25',
 'pistacking_HIS_41',
 'saltbridge_GLU_166']
len(ref_i)

26

In [9]:
mpro_crystal_structures_interactions = {'hacceptor_THR_25_8',
                                         'hdonor_THR_25_8',
                                         'hydrophobic_THR_25',
                                         'hacceptor_HIS_41_8',
                                         'hydrophobic_HIS_41',
                                         'pistacking_HIS_41',
                                         'hacceptor_CYS_44_8',
                                         'hydrophobic_PHE_140',
                                         'hacceptor_ASN_142_8',
                                         'hdonor_ASN_142_7',
                                         'hdonor_GLY_143_7',
                                        'hacceptor_SER_144_8',
                                        'hdonor_SER_144_7',
                                        'hdonor_SER_144_8',
                                        'hdonor_CYS_145_7',
                                        'hacceptor_HIS_163_7',
                                        'hydrophobic_MET_165',
                                        'hacceptor_GLU_166_8',
                                        'hdonor_GLU_166_7',
                                        'hdonor_GLU_166_8',
                                        'hydrophobic_GLU_166',
                                        'saltbridge_GLU_166',
                                        'hydrophobic_PRO_168',
                                        'hydrophobic_ASP_187',
                                        'hacceptor_GLN_189_8',
                                        'hdonor_GLN_189_7',
                                        'hydrophobic_GLN_189'}
set(mpro_crystal_structures_interactions)

{'hacceptor_ASN_142_8',
 'hacceptor_CYS_44_8',
 'hacceptor_GLN_189_8',
 'hacceptor_GLU_166_8',
 'hacceptor_HIS_163_7',
 'hacceptor_HIS_41_8',
 'hacceptor_SER_144_8',
 'hacceptor_THR_25_8',
 'hdonor_ASN_142_7',
 'hdonor_CYS_145_7',
 'hdonor_GLN_189_7',
 'hdonor_GLU_166_7',
 'hdonor_GLU_166_8',
 'hdonor_GLY_143_7',
 'hdonor_SER_144_7',
 'hdonor_SER_144_8',
 'hdonor_THR_25_8',
 'hydrophobic_ASP_187',
 'hydrophobic_GLN_189',
 'hydrophobic_GLU_166',
 'hydrophobic_HIS_41',
 'hydrophobic_MET_165',
 'hydrophobic_PHE_140',
 'hydrophobic_PRO_168',
 'hydrophobic_THR_25',
 'pistacking_HIS_41',
 'saltbridge_GLU_166'}

In [10]:
set1 = mpro_crystal_structures_interactions
set2 = set(ref_i)
# calculate differences
difference_set1_set2 = set1 - set2
difference_set2_set1 = set2 - set1

# print differences
print("Unique to set 1:", difference_set1_set2)
print("Unique to set 2:", difference_set2_set1)


NameError: name 'ref_i' is not defined

In [15]:
i_dict = {}
struc_dir = '.'
for i in [1,5,6,10]:
    i_dict[str(i)] = itrns, _ = plip_mpro_score(struc_dir+f'/{i}_complexh.pdb')

i_dict

FileNotFoundError: [Errno 2] No such file or directory: './1_complexh.pdb'

In [22]:
i_dict = {}
struc_dir = '/home/cree/code/gal/cs50k/enamine_study/sars-cov-2-main-protease-al-study-plip/plip_tests'
for i in ['5RGI', '5RF7', '5RF2', '5RG1', '5RGK']:
    i_dict[str(i)] = itrns, _ = plip_mpro_score(struc_dir+f'/{i}_h.pdb')

i_dict

{'5RGI': (['hydrophobic_GLU_166',
   'hdonor_HIS_163_7',
   'hdonor_SER_144_8',
   'hdonor_GLY_143_7',
   'hdonor_SER_144_7',
   'hdonor_CYS_145_7'],
  0.17857142857142858),
 '5RF7': (['hydrophobic_MET_49',
   'hydrophobic_GLU_166',
   'hdonor_GLU_166_7',
   'hdonor_HIS_163_7'],
  0.06896551724137931),
 '5RF2': (['hydrophobic_THR_25',
   'hdonor_THR_25_8',
   'hacceptor_CYS_44_8',
   'hacceptor_HIS_41_8'],
  0.14814814814814814),
 '5RG1': (['hdonor_SER_144_8',
   'hdonor_HIS_163_7',
   'hdonor_GLU_166_7',
   'hacceptor_SER_144_8',
   'hacceptor_GLU_166_8'],
  0.14285714285714285),
 '5RGK': (['hydrophobic_ASN_142',
   'hydrophobic_GLU_166',
   'hdonor_ASN_142_7',
   'hdonor_HIS_163_7',
   'hacceptor_ASN_142_8'],
  0.10344827586206896)}

In [32]:
a_dict = {}
a_dict[str('10')] = itrns, _ = plip_mpro_score('../10_altrec.pdb')
a_dict

{'10': (['hydrophobic_GLU_166',
   'hdonor_GLU_166_7',
   'hdonor_GLY_143_7',
   'hdonor_SER_144_7',
   'hdonor_CYS_145_7'],
  0.18518518518518517)}

In [34]:
mpro_xstal = {'hacceptor_THR_25_8',
                                         'hdonor_THR_25_8',
                                         'hydrophobic_THR_25',
                                         'hacceptor_HIS_41_8',
                                         'hydrophobic_HIS_41',
                                         'pistacking_HIS_41',
                                         'hacceptor_CYS_44_8',
                                         'hydrophobic_PHE_140',
                                         'hacceptor_ASN_142_8',
                                         'hdonor_ASN_142_7',
                                         'hdonor_GLY_143_7',
                                        'hacceptor_SER_144_8',
                                        'hdonor_SER_144_7',
                                        'hdonor_SER_144_8',
                                        'hdonor_CYS_145_7',
                                        'hacceptor_HIS_163_7',
                                        'hydrophobic_MET_165',
                                        'hacceptor_GLU_166_8',
                                        'hdonor_GLU_166_7',
                                        'hdonor_GLU_166_8',
                                        'hydrophobic_GLU_166',
                                        'saltbridge_GLU_166',
                                        'hydrophobic_PRO_168',
                                        'hydrophobic_ASP_187',
                                        'hacceptor_GLN_189_8',
                                        'hdonor_GLN_189_7',
                                        'hydrophobic_GLN_189'}

In [35]:
len(mpro_xstal)

27

In [68]:
a_dict = {}
a_dict[str('matt')] = itrns, _ = plip_mpro_score('../matt_h.pdb')
a_dict

{'matt': (['hydrophobic_HIS_41',
   'hydrophobic_GLU_166',
   'hydrophobic_GLN_189',
   'hdonor_GLU_166_7',
   'hdonor_HIS_163_7',
   'hdonor_GLY_143_7',
   'hdonor_CYS_145_7',
   'hacceptor_ASN_142_8',
   'pistacking_HIS_41'],
  0.2857142857142857)}